In [1]:
# Copyright 2024 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Define Constants

In [2]:
USER_ID = ""  # @param {type:"string"}
DEMO_PROJECT = ""
BIGQUERY_DATASET = "minigolf"
BIGQUERY_TRACKING_TABLE = "tracking"
GS_BUCKET = ""
BACKGROUND_IMAGE_BUCKET = ""

### Import necessary Libraries

In [4]:
import pandas as pd
from google.cloud import bigquery
import seaborn as sns

import os

import matplotlib.pyplot as plt
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models
from IPython.display import display, Markdown

If you have `AttributeError: module 'bigframes' has no attribute 'dataframe'` after executing the previous cell,

execute `!pip install vertexai`. It requires to restart the runtime.

### Unveiling the Mini-Golf Stats

Now, let's dive into the numbers behind the swings! This section analyzes the performance data from all players, giving us insights into how everyone fared on the mini-golf course.

We calculate some key statistics:

* **Average Number of Shots**: This tells us the average number of shots taken by all players. It gives us a general idea of the course's difficulty.
* **Median Number of Shots**: This represents the middle value when all players' shot counts are ordered. It's less affected by outliers (extremely high or low scores) than the average.

Finally, we present a **bar chart** visualizing the distribution of shots taken by all players. This gives us a quick overview of how many players took a certain number of shots.

So, take a look at the stats and see how you stack up against the competition!

In [ ]:
# Configure BigQuery client
client = bigquery.Client()
query = f"SELECT * FROM {DEMO_PROJECT}.{BIGQUERY_DATASET}.{BIGQUERY_TRACKING_TABLE}"

df = client.query(query).to_dataframe()
last_frame_per_user = df.groupby('user_id')['frame_number'].transform(max)
df_filtered = df[df['frame_number'] == last_frame_per_user]
user_shot_counts = df_filtered.groupby('user_id')['shot_number'].first()
user_shot_counts = user_shot_counts[user_shot_counts > 0]
shot_number_freq = user_shot_counts.value_counts()

# Selected user's number of shots
num_users = df['user_id'].nunique()
user_shots = user_shot_counts.get(USER_ID, 0)
average_shots_per_user = user_shot_counts.mean()
median_shots_per_user = user_shot_counts.median()

display(Markdown(f"### There are {num_users} users played so far."))
display(Markdown(f"### User {USER_ID}'s number of shots: {user_shots}"))
display(Markdown(f"### Average number of shots: {average_shots_per_user:.2f}"))
display(Markdown(f"### Median number of shots: {median_shots_per_user:.2f}"))

# Transform the 'shot_number_freq' Series into a DataFrame
table_data = shot_number_freq.to_frame().reset_index() 
table_data.columns = ['Number of Shots', 'Number of Users']
display(table_data)

# Plot a bar chart
plt.xlim(0, 10)
plt.bar(shot_number_freq.index, shot_number_freq.values, color='#4285F4')
plt.xlabel('Number of Shots')
plt.ylabel('Number of Users')
plt.title('Distribution of Number of Shots per User')
plt.show()


### Visualizing Your Mini-Golf Journey

This graph provides a bird's-eye view of your mini-golf adventure, showing exactly how the ball traveled across the course. Each cluster of colorful dots represents a specific moment in time – the spot where the ball came to rest after each shot. It's like seeing a snapshot of your game, frozen in time!

The colors help us follow the sequence of your shots. The first shot is represented by one color, the second shot by another, and so on. This allows us to easily trace the progression of your game from start to finish.

So, take a moment to explore the graph and relive your mini-golf experience. It's a visual representation of your skill and the exciting journey of that little white ball!



In [ ]:
client = bigquery.Client()
query = f"""
SELECT * FROM `{DEMO_PROJECT}.{BIGQUERY_DATASET}.{BIGQUERY_TRACKING_TABLE}`
WHERE user_id = "{USER_ID}"
"""

df = client.query(query).to_dataframe()
df['shot_number_str'] = df['shot_number'].astype(str)

file_name = f"BG_{USER_ID}.jpg"
image_uri = f"gs://{BACKGROUND_IMAGE_BUCKET}/{file_name}"
local_image_path = '/tmp/downloaded_image.jpg'
os.system(f'gsutil cp {image_uri} {local_image_path}')
img = plt.imread(local_image_path)

fig, ax = plt.subplots(figsize=(15, 7))
color_palette = sns.color_palette(
    'tab10',
    n_colors=len(df['shot_number_str'].unique())
)

unique_user_ids = df['user_id'].unique()
sns.scatterplot(
    data=df, x='x', y='y', hue='shot_number_str',
    legend=False, palette=color_palette, size=1, ax=ax
)

# invert y-axis
plt.gca().invert_yaxis()

ax.imshow(img)
plt.show()


### Gemini: The AI Sportscaster

Imagine Gemini as a super-smart assistant who watches your mini-golf game. It can't see you swing the club, but it follows the ball using a video camera, acting like a data detective. Gemini analyzes the video frame-by-frame, tracking the ball's path and how close it gets to the hole.

Once Gemini understands the game, it transforms into a real announcer, describing each shot with excitement and flair. But instead of a human voice, it uses its artificial intelligence to generate the commentary. This is possible because Gemini has been trained on a massive amount of sportscasting data, learning the language, style, and energy that announcers bring to the game.

The best part? Gemini does all this in real-time! As soon as you finish a hole, it instantly generates commentary, providing insights without any manual effort. It's like having a combination of a data analyst and a sports announcer, all powered by AI. Gemini takes the "what" (video data) and turns it into the "how" (play-by-play) using its knowledge of sportscasting gleaned from vast amounts of training data.

So, whether you're a guest enjoying the game or a presenter explaining the technology, think of Gemini as your AI sportscaster, bringing a unique and insightful perspective to the mini-golf experience!


In [ ]:
URI = f"gs://{GS_BUCKET}/{USER_ID}.MP4"
VIDEO = Part.from_uri(uri=URI, mime_type="video/mp4")

PRE_DEFINED_TEXT = """You are a professional golf announcer and you must broadcast the match a formal and informative tone. You should use the following context.
- The match is 'Google Cloud Next - Minigolf Championship final', and the venue is Mandalay Bay, Las Vegas.
- The competitor already completed the game and if the player complete this hole within three shots, the player wins.
- If the hole is completed over four shots, THE COMPETITOR WINS.
- Even though the competitor wins, you must broadcast the game until the player finishes the last shot.
- You should not mention anything about the players' appearances or personal lives.
- The broadcast must be done in colloquial language and no additional text other than the announcer's comments (e.g., cheers from the audience must not be included).
- The course is a rectangle measuring 7 feet by 20 feet, and there are no obstacles or slopes on the course.
- The video is recorded in 60 fps.
- Describe each shots in detail.
- This text will be shown in a markdown format, so make sure add some markdowns as you emphasize.
"""



def generate_commentary(df: pd.DataFrame) -> str:
    filtered_df = df[df['shot_number'] > 0]

    # Group by shot_number
    grouped = filtered_df.groupby('shot_number')
    first_last_rows = []

    # Iterate over each group
    for shot_number, group_df in grouped:
        # Get the first and last rows for the current shot_number
        first_row = group_df.head(1)
        last_row = group_df.tail(1)

        # Append the first and last rows to the list
        first_last_rows.append(first_row)
        first_last_rows.append(last_row)

    # Concatenate the list of DataFrames into a single DataFrame
    result_df = pd.concat(first_last_rows)

    shot_details = ""
    for _, row in result_df.iterrows():
        shot_number = row['shot_number']
        if row['is_moving']:
            shot_details += f"- Shot {shot_number} started from {row['distance']:.2f} pixels from the hole\n"
        else:
            shot_details += f"- Shot {shot_number} stopped {row['distance']:.2f} pixels from the hole\n"

    # Check if last shot distance is less than 50 (made a hole-in)
    result = "didn't make" if df['distance'].iloc[-1] > 50 else "made"

    commentary = f"""
    - Here's the analytics of each shot extracted from the video. Use it as a reference:
    {shot_details}
    - The ball {result.upper()} the hole after the shot number {shot_number}.
    - The measurement of the distance is in pixels. The distance is measured from the center of the ball to the center of the hole.
    - DO NOT USE pixels as a unit of measurement. USE ONLY feet and yards. Convert pixel to feet and yards appropriately.
    - After the final shot, if the remaining pixel is less than 50 pixel, then consider it as a hole-in.
    """
    return commentary

def generate():
    vertexai.init(project=DEMO_PROJECT, location="us-central1")
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    responses = model.generate_content(
        [VIDEO, PRE_DEFINED_TEXT, generate_commentary(df)],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )
    display(Markdown((responses.text)))


generation_config = {
    "max_output_tokens": 8192,
    "temperature": 2,
    "top_p": 0.4,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generate()


### Data Privacy and Deletion:

To protect your privacy, the video of your mini-golf game will be deleted from cloud storage after a day. 
The raw footage was only used to analyze ball positions for generating commentary and insights.

Rest assured, your privacy is our top priority. Only the calculated data (ball and hole positions) will be retained for further analysis.

